In [39]:
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import calendar
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

with open('api_key.txt', 'r') as f:
    API_KEY = f.read().strip()

year = 2026

# === CONFIG ===
SHEET_ID = '1Er54lmX1jBCbjQWkcFjWX6hp0WsoTiSqeb8U9SNIReo'
RANGE = f'{year}!A1:Z215'

# === SETUP ===
service = build('sheets', 'v4', developerKey=API_KEY)
sheet = service.spreadsheets()

# === STEP 1: Get full sheet grid with formatting ===
result = sheet.get(
    spreadsheetId=SHEET_ID,
    ranges=[RANGE],
    includeGridData=True
).execute()

grid = result['sheets'][0]['data'][0]['rowData']

In [40]:
# Step 1: Find all month labels (like 'JANUARY', 'FEBRUARY', etc.)
month_positions = {}  # (row_idx, col_idx) => month_num
month_names = {name.upper(): i for i, name in enumerate(calendar.month_name) if name}

for row_idx, row in enumerate(grid):
    for col_idx, cell in enumerate(row.get('values', [])):
        val = cell.get('formattedValue', '')
        val_clean = val.strip().upper()
        if val_clean in month_names:
            month_num = month_names[val_clean]
            month_positions[(row_idx, col_idx)] = month_num

# Step 2: For each day-number cell, find the nearest month label *above and to the right*
date_map = {}  # (row_idx, col_idx) -> datetime

for row_idx, row in enumerate(grid):
    for col_idx, cell in enumerate(row.get('values', [])):
        val = cell.get('formattedValue', '')
        if not val.isdigit():
            continue
        day = int(val)

        # Find the closest month label above and to the right
        best_month = None
        best_distance = float('inf')

        for (m_row, m_col), month_num in month_positions.items():
            if m_row < row_idx and m_col <= col_idx:
                dist = (row_idx - m_row) + (col_idx - m_col)
                if dist < best_distance:
                    best_distance = dist
                    best_month = month_num

        if best_month is not None:
            try:
                date = datetime(year, best_month, day)
                date_map[(row_idx, col_idx)] = date
            except ValueError:
                pass  # skip invalid dates like Feb 30


In [41]:
events = []
merged_ranges = result['sheets'][0].get('merges', [])
visited = set()

# Helper to find merged range for a given cell
def get_merged_range(row, col):
    for mr in merged_ranges:
        if (
            mr['startRowIndex'] <= row < mr['endRowIndex']
            and mr['startColumnIndex'] <= col < mr['endColumnIndex']
        ):
            return mr
    return None

for row_idx, row in enumerate(grid):
    for col_idx, cell in enumerate(row.get('values', [])):
        if (row_idx, col_idx) in visited:
            continue

        val = cell.get('formattedValue')
        if not val or val.strip().isdigit():
            continue
        link = cell.get('hyperlink', '')

        # === Get event start date and event type color from the date cell above ===
        event_date = None
        event_type = 'Other'

        for r2 in range(row_idx, 0, -1):
            if (r2, col_idx) in date_map:
                event_date = date_map[(r2, col_idx)]

                # Try to read the background color from the date cell
                try:
                    date_cell = grid[r2]['values'][col_idx]  # <-- correct path
                    color = date_cell.get('effectiveFormat', {}).get('backgroundColor', {})
                    r = color.get('red', 0)
                    g = color.get('green', 0)
                    b = color.get('blue', 0)

                    if r > 0.8 and g < 0.5:
                        event_type = 'GBHL100'
                    elif r > 0.8 and g > 0.8:
                        event_type = 'GBHL90'
                    elif b > 0.8:
                        event_type = 'GBHL80'
                    else:
                        event_type = 'Other'
                except Exception as e:
                    print(f"Warning: Failed to read event type color from ({r2},{col_idx}): {e}")

                break

        if not event_date:
            continue

        # Parse event text
        lines = val.strip().split('\n')
        if len(lines) < 3:
            continue

        event_name = lines[0].strip()
        organizer = lines[1].strip()
        region_loc = lines[2].strip().strip('[]')
        region, location = region_loc.split(' - ') if ' - ' in region_loc else ('Unknown', region_loc)

        # Determine merged range (if any)
        merged = get_merged_range(row_idx, col_idx)
        if merged:
            end_col = merged['endColumnIndex'] - 1
            # Mark all merged cells as visited
            for r in range(merged['startRowIndex'], merged['endRowIndex']):
                for c in range(merged['startColumnIndex'], merged['endColumnIndex']):
                    visited.add((r, c))
        else:
            end_col = col_idx
            visited.add((row_idx, col_idx))

        # Get end date from the right-most merged cell's column
        end_date = event_date
        for r2 in range(row_idx, 0, -1):
            if (r2, end_col) in date_map:
                end_date = date_map[(r2, end_col)]
                break

        # === Extract format (Singles / Doubles) from the row below ===
        format_type = 'Unknown'

        # Determine the row below the event block
        format_row_idx = (merged['endRowIndex'] if merged else row_idx + 1)

        # Use the first column of the merged block if available, else same col
        format_col_idx = merged['startColumnIndex'] if merged else col_idx

        # Safely access the format cell
        if format_row_idx < len(grid):
            format_row = grid[format_row_idx]
            if format_col_idx < len(format_row.get('values', [])):
                format_cell = format_row['values'][format_col_idx]
                format_val = format_cell.get('formattedValue', '').strip().lower()
                if 'double' in format_val:
                    format_type = 'Doubles'
                elif 'single' in format_val:
                    format_type = 'Singles'
                elif 'team' in format_val:
                    format_type = 'Team'
                # Assume missing formats are singles events
                else:
                    format_type = 'Singles'

        event = {
            'start_date': event_date.strftime('%Y-%m-%d'),
            'end_date': end_date.strftime('%Y-%m-%d'),
            'event_name': event_name,
            'organizer': organizer,
            'region': region,
            'location': location,
            'format': format_type,
            'event_type': event_type,
            'link': link
        }

        events.append(event)


In [42]:
# === STEP 4: Show / Save Results ===
df = pd.DataFrame(events)
print(df.head())

   start_date    end_date                event_name        organizer region  \
0  2026-01-02  2026-01-02           A New Adventure        Matt King     SW   
1  2026-01-03  2026-01-04        Into The West 2026        Matt King     SW   
2  2026-03-28  2026-03-01  Defence of North Bristol    David Clubley     SW   
3  2026-03-28  2026-03-01    Bonds of Fellowship VI    Callum Slater     SE   
4  2026-03-28  2026-03-01     Honour The Allegiance  Natalie Pearson  N.Eng   

    location   format event_type link  
0    Cardiff  Singles     GBHL90       
1    Cardiff  Singles    GBHL100       
2    Bristol  Singles     GBHL90       
3    Havantl  Singles     GBHL90       
4  Sheffield  Singles     GBHL90       


In [43]:
df.event_type.value_counts()

event_type
GBHL90     51
GBHL80     39
GBHL100    20
Name: count, dtype: int64

In [44]:
print(df.tail())

     start_date    end_date                        event_name  \
105  2026-10-24  2026-10-24           Great Deeds of War 2026   
106  2026-10-25  2026-10-25           Battle for Middle Earth   
107  2026-10-31  2026-10-01  The Crownless Shall Be King 2026   
108  2026-11-28  2026-11-29           He Eats It By The Block   
109  2026-11-28  2026-11-28            Grand Anglian Alliance   

           organizer region      location   format event_type link  
105   Rob Lainchbury      C    Birmingham  Singles     GBHL90       
106    Chris Jackson     SE    Eastbourne  Singles     GBHL90       
107     Tom Culleton     SE  High Wycombe  Singles     GBHL90       
108  Jack Darlington      C     Leicester  Singles    GBHL100       
109     James Palmer     SE       Woolpit  Singles     GBHL80       


In [45]:
old_df = pd.read_csv('data/gbhl_events.csv')

In [46]:
old_df = old_df[["event_name", "start_date", "lat", "lon"]]

In [47]:
df = df.merge(old_df, on=["event_name", "start_date"], how="left")

In [48]:
# Manual fixes on location typos
df["location"].replace("Abergevanny", "Abergavenny", inplace=True)
df["location"].replace("Bedwroth", "Bedworth", inplace=True)

In [49]:
# Create a location string for geocoding
df['location_str'] = df['location'] + ', UK'

# Set up Nominatim geocoder
geolocator = Nominatim(user_agent="gbhl-event-locator")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # be polite!

# Apply geocoding
def get_coords(place):
    try:
        loc = geocode(place)
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
    except:
        pass
    return pd.Series([None, None])

# Find rows where lat or lon is null
missing_coords = df[df['lat'].isnull() | df['lon'].isnull()]

In [50]:
missing_coords

,start_date,end_date,event_name,organizer,region,location,format,event_type,link,lat,lon,location_str
0,2026-01-02,2026-01-02,A New Adventure,Matt King,SW,Cardiff,Singles,GBHL90,,NaN,NaN,"Cardiff, UK"
1,2026-01-03,2026-01-04,Into The West 2026,Matt King,SW,Cardiff,Singles,GBHL100,,NaN,NaN,"Cardiff, UK"
2,2026-03-28,2026-03-01,Defence of North Bristol,David Clubley,SW,Bristol,Singles,GBHL90,,NaN,NaN,"Bristol, UK"
3,2026-03-28,2026-03-01,Bonds of Fellowship VI,Callum Slater,SE,Havantl,Singles,GBHL90,,NaN,NaN,"Havantl, UK"
4,2026-03-28,2026-03-01,Honour The Allegiance,Natalie Pearson,N.Eng,Sheffield,Singles,GBHL90,,NaN,NaN,"Sheffield, UK"
...,...,...,...,...,...,...,...,...,...,...,...,...
105,2026-10-24,2026-10-24,Great Deeds of War 2026,Rob Lainchbury,C,Birmingham,Singles,GBHL90,,NaN,NaN,"Birmingham, UK"
106,2026-10-25,2026-10-25,Battle for Middle Earth,Chris Jackson,SE,Eastbourne,Singles,GBHL90,,NaN,NaN,"Eastbourne, UK"
107,2026-10-31,2026-10-01,The Crownless Shall Be King 2026,Tom Culleton,SE,High Wycombe,Singles,GBHL90,,NaN,NaN,"High Wycombe, UK"
108,2026-11-28,2026-11-29,He Eats It By The Block,Jack Darlington,C,Leicester,Singles,GBHL100,,NaN,NaN,"Leicester, UK"


In [ ]:
if not missing_coords.empty:
    # Apply geocoding function just to these rows
    missing_coords[['lat', 'lon']] = missing_coords['location_str'].apply(get_coords)

    # Update the original dataframe only for those indices
    df.loc[missing_coords.index, ['lat', 'lon']] = missing_coords[['lat', 'lon']]

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Winchester, UK',), **{}).
Traceback (most recent call last):
  File "c:\Users\samgu\anaconda3\envs\py3.11\Lib\site-packages\urllib3\connectionpool.py", line 467, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\samgu\anaconda3\envs\py3.11\Lib\site-packages\urllib3\connectionpool.py", line 462, in _make_request
    httplib_response = conn.getresponse()
                       ^^^^^^^^^^^^^^^^^^
  File "c:\Users\samgu\anaconda3\envs\py3.11\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "c:\Users\samgu\anaconda3\envs\py3.11\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samgu\anaconda3\envs\py3.11\Lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^

In [ ]:
def classify_event_duration(start_str, end_str):
    start_date = datetime.strptime(start_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_str, '%Y-%m-%d')
    start_dow = start_date.weekday()  # 0=Monday ... 5=Saturday ... 6=Sunday
    end_dow = end_date.weekday()

    if start_dow == 5 and end_dow == 5:
        return "Saturday"
    elif start_dow == 6 and end_dow == 6:
        return "Sunday"
    elif start_dow == 5 and end_dow == 6:
        return "Weekend"
    else:
        return "Other"

# Apply to dataframe
df['event_duration'] = df.apply(
    lambda row: classify_event_duration(row['start_date'], row['end_date']),
    axis=1
)


In [ ]:
df

In [ ]:
df.to_csv("data/gbhl_events.csv", index=False)

In [ ]:
df.to_json('data/events.json', orient='records')